In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc,os,sys

sns.set_style('darkgrid')
pd.options.display.float_format = '{:,.3f}'.format

print(os.listdir("../input"))

# Load data

In [ ]:
%%time
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

print(train.shape, test.shape)

In [ ]:
for c in train.columns:
    if c not in test.columns: print(c)

# Data analysis

In [ ]:
train.head()

In [ ]:
null_cnt = train.isnull().sum().sort_values()
print('null count:', null_cnt[null_cnt > 0])

In [ ]:
train['target'].value_counts().to_frame().plot.bar()

# Feature engineering

In [ ]:
all_data = train.append(test, sort=False).reset_index(drop=True)
del train, test
gc.collect()

all_data.head()

In [ ]:
# drop constant column
constant_column = [col for col in all_data.columns if all_data[col].nunique() == 1]
print('drop columns:', constant_column)
all_data.drop(constant_column, axis=1, inplace=True)

In [ ]:
corr_matrix = all_data.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [c for c in upper.columns if any(upper[c] > 0.95)]
del upper

drop_column = all_data.columns[to_drop]
print('drop columns:', drop_column)
#all_data.drop(drop_column, axis=1, inplace=True)

In [ ]:
cols = [col for col in all_data.columns if col not in ['id','target']]
for i, t in all_data.loc[:, cols].dtypes.iteritems():
    if t == object:
        print(i)
        all_data[i] = pd.factorize(all_data[i])[0]

## scaling

In [ ]:
from sklearn import preprocessing

numcols = all_data.drop(['id','target'],axis=1).select_dtypes(include='number').columns.values
scaler = preprocessing.StandardScaler()
all_data.loc[:,numcols] = scaler.fit_transform(all_data[numcols])

# Prepare

In [ ]:
X_train = all_data[all_data['target'].notnull()].reset_index(drop=True)
X_test = all_data[all_data['target'].isnull()].drop(['target'], axis=1).reset_index(drop=True)
del all_data
gc.collect()

# drop ID_code
X_train.drop(['id'], axis=1, inplace=True)
X_test_ID = X_test.pop('id')

Y_train = X_train.pop('target')

print(X_train.shape, X_test.shape)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_train)
ev_ratio = pca.explained_variance_ratio_
ev_ratio = np.hstack([0,ev_ratio.cumsum()])

plt.xlabel('components')
plt.plot(ev_ratio)
plt.show()

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=2, init='k-means++', max_iter=3000, random_state=42)
X_train_km = km.fit_predict(X_train)
X_test_km = km.predict(X_test)

km_data = pd.DataFrame({'KMeans':X_train_km, 'target':Y_train})
sns.countplot(x='KMeans', hue='target', palette='Set1', data=km_data)

In [ ]:
_='''
from sklearn.feature_selection import SelectKBest, f_classif

feat = SelectKBest(f_classif, k=150)
feat.fit(X_train, Y_train)

X_train = feat.transform(X_train)
X_test = feat.transform(X_test)
'''

In [ ]:
from imblearn.over_sampling import SMOTE,ADASYN

#sm = SMOTE(random_state=42)
#sm = SMOTE(kind='svm',random_state=42)
#sm = SMOTE(kind='borderline1',random_state=42)
sm = ADASYN(random_state=42)
X_train, Y_train = sm.fit_sample(X_train, Y_train)
X_train = pd.DataFrame(X_train, columns=X_test.columns)
print(X_train.shape)

# Predict

In [ ]:
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE, RFECV

In [ ]:
splits = 10
folds = RepeatedStratifiedKFold(n_splits=splits, n_repeats=20, random_state=42)
oof_preds = np.zeros(X_train.shape[0])
sub_preds = np.zeros(X_test.shape[0])

for fold_, (trn_, val_) in enumerate(folds.split(X_train, Y_train)):
    trn_x, trn_y = X_train.iloc[trn_], Y_train[trn_]
    val_x, val_y = X_train.iloc[val_], Y_train[val_]

    '''
    clf = LogisticRegression(C=0.3, max_iter=1000, class_weight='balanced', 
            penalty='l1', solver='liblinear', random_state=42)
    #model = RFE(clf, 25, step=1)
    model = RFECV(clf, step=1, cv=(splits - 1))
    model.fit(trn_x, trn_y)
    oof_preds[val_] = model.predict_proba(val_x)[:,1]
    sub_preds += model.predict_proba(X_test)[:,1] / splits / 20 #folds.n_splits
    '''
    clf = Lasso(alpha=0.03, tol=0.01, selection='random', random_state=42)
    model = RFE(clf, 20, step=1)
    #model = RFECV(clf, step=1, cv=(splits - 1))
    model.fit(trn_x, trn_y)
    
    preds = model.predict(val_x)
    preds = preds / (-0.78 * preds + 1.78)
    oof_preds[val_] = preds.clip(0, 1)
    sub_preds += model.predict(X_test).clip(0, 1) / splits / 20 #folds.n_splits

In [ ]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(Y_train, oof_preds)
auc = metrics.auc(fpr, tpr)

plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
plt.legend()
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

# Submit

In [ ]:
submission = pd.DataFrame({
    'id': X_test_ID,
    'target': sub_preds
})
submission.to_csv("submission.csv", index=False)

In [ ]:
print(submission['target'].sum() / len(submission))
submission['target'].hist(bins=25, alpha=0.6)

In [ ]:
submission.head()